# Settings

In [ ]:
import platform  

def get_device():
    """
    Function to determine the device type based on the node name.
    It uses a dictionary to map node names to device types.

    :return: device type as a string
    :raises Exception: if node name is not found in the device_map dictionary
    """
    device_map = {
        "PC-Cristian": "cuda",
        "Dell-G5-15-Alexios": "cuda",
        "MacBook-Pro-di-Cristian.local": "mps",
        "MacBookProDiGrazia": "cpu",
        "DESKTOP-RQVK8SI":"cuda",
        "MacBook-Pro.station":"mps"
    }

    try:
        return device_map[platform.uname().node]
    except KeyError:
        raise Exception("Node name not found. Please add your node name and its corresponding device to the dictionary.")

device = get_device()

In [ ]:
import os
from project_paths import *

def delete_ds_store_files(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Deleted {file_path}")

# Specifica la cartella principale in cui cercare i file .DS_Store
folder_path = data_folder_path

delete_ds_store_files(folder_path)

# Create YOLO dataset folder

In [ ]:
import os, shutil, random

def create_yolo_datset(splitted_frames_path, splitted_gt_path, yolo_dataset_path, num_frames_per_video=10):

    # folders
    # if yolo dataset folder, doesn't exist, create it
    if not os.path.exists(yolo_dataset_path):
        os.makedirs(yolo_dataset_path)  

    # create the yolo dataset structure, we create a folder for train and a folder for val
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val'))

    # create the two inner folders: one for fire and one for no fire
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train', 'fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train', 'fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'train', 'no_fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'train', 'no_fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val', 'fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val', 'fire'))
    if not os.path.exists(os.path.join(yolo_dataset_path, 'val', 'no_fire')):
        os.makedirs(os.path.join(yolo_dataset_path, 'val', 'no_fire'))

    # now we copy the images in the right folder
    # we have to copy the images in the train folder and in the val folder
    # for each folder in SPLITTED_FRAMES_DATASET/TRAINING_SET/0 we randomly sample num_frames_per_video frames and put them in dataset/train/no_fire
    # for each folder in SPLITTED_FRAMES_DATASET/VALIDATION_SET/0 we randomly sample num_frames_per_video frames and put them in dataset/val/no_fire

    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        for _ in range (num_frames_per_video):
            # randomly sample a frame
            frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder)))
            while frame in chosen_frames:
                frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder)))
            chosen_frames.append(frame)
            # copy the frame in the right folder
            shutil.copy(os.path.join(splitted_frames_path, 'TRAINING_SET','0', folder, frame), os.path.join(yolo_dataset_path, 'train', 'no_fire', frame))
            # rename the frame
            os.rename(os.path.join(yolo_dataset_path, 'train', 'no_fire', frame), os.path.join(yolo_dataset_path, 'train', 'no_fire', str(count)+'.jpg'))
            count += 1
        
    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        for _ in range (num_frames_per_video):
            # randomly sample a frame
            frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder)))
            while frame in chosen_frames:
                frame = random.choice(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder)))
            chosen_frames.append(frame)
            # copy the frame in the right folder
            shutil.copy(os.path.join(splitted_frames_path, 'VALIDATION_SET','0', folder, frame), os.path.join(yolo_dataset_path, 'val', 'no_fire', frame))
            # rename the frame
            os.rename(os.path.join(yolo_dataset_path, 'val', 'no_fire', frame), os.path.join(yolo_dataset_path, 'val', 'no_fire', str(count)+'.jpg'))
            count += 1

    # for each folder in SPLITTED_FRAMES_DATASET/TRAINING_SET/1 we randomly sample num_frames_per_video frames starting from the frame indicated in the annotation file
    # and put them in dataset/train/fire
    # for each folder in SPLITTED_FRAMES_DATASET/VALIDATION_SET/1 we randomly sample num_frames_per_video frames starting from the frame indicated in the annotation file
    # and put them in dataset/val/fire

    count = 0    
    for folder in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        # get the annotation file corresponding to the folder
        annotation_file = folder.replace("mp4", "rtf")
        # open the annotation file
        with open(os.path.join(splitted_gt_path, 'TRAINING_SET', '1', annotation_file), 'r') as f:
            # get the frame in which the fire starts
            start_frame = int(f.readline().split(',')[0])
            # randomly sample a frame whose number starts from start_frame
            available_frames = []
            for frame in os.listdir(os.path.join(splitted_frames_path, 'TRAINING_SET','1', folder)):
                if int(frame.split('.')[0])>=start_frame:
                    available_frames.append(frame)
            frame = random.choice(available_frames)
            while frame in chosen_frames:
                frame = random.choice(available_frames)
            chosen_frames.append(frame)
            # copy the frame in the right folder
            shutil.copy(os.path.join(splitted_frames_path, 'TRAINING_SET','1', folder, frame), os.path.join(yolo_dataset_path, 'train', 'fire', frame))
            # rename the frame
            os.rename(os.path.join(yolo_dataset_path, 'train', 'fire', frame), os.path.join(yolo_dataset_path, 'train', 'fire', str(count)+'.jpg'))
            count += 1

    count = 0
    for folder in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1')):
        tot_frames = len(os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1')))
        if tot_frames < num_frames_per_video:
            num_frames_per_video = tot_frames
        chosen_frames = []
        # get the annotation file corresponding to the folder
        annotation_file = folder.replace("mp4", "rtf")
        # open the annotation file
        with open(os.path.join(splitted_gt_path, 'VALIDATION_SET', '1', annotation_file), 'r') as f:
            # get the frame in which the fire starts
            start_frame = int(f.readline().split(',')[0])
            # randomly sample a frame whose number starts from start_frame
            available_frames = []
            for frame in os.listdir(os.path.join(splitted_frames_path, 'VALIDATION_SET','1', folder)):
                if int(frame.split('.')[0])>=start_frame:
                    available_frames.append(frame)
            frame = random.choice(available_frames)
            while frame in chosen_frames:
                frame = random.choice(available_frames)
            chosen_frames.append(frame)
            # copy the frame in the right folder
            shutil.copy(os.path.join(splitted_frames_path, 'VALIDATION_SET','1', folder, frame), os.path.join(yolo_dataset_path, 'val', 'fire', frame))
            # rename the frame
            os.rename(os.path.join(yolo_dataset_path, 'val', 'fire', frame), os.path.join(yolo_dataset_path, 'val', 'fire', str(count)+'.jpg'))
            count += 1

create_yolo_datset(splitted_frames_path, splitted_annotations_path, yolo_folder_path, num_frames_per_video=3)


# Preprocess images

In [ ]:
# open all the images in yolo_dataset and resize them to 224x224
from PIL import Image
from tqdm import tqdm
import os

def resize_images(dataset_path, img_size=224):

    folders = ['train', 'val']
    subfolders = ['fire', 'no_fire']

    for folder in folders:
        for subfolder in subfolders:
            for image in tqdm(os.listdir(os.path.join(dataset_path, folder, subfolder))):
                img = Image.open(os.path.join(dataset_path, folder, subfolder, image))
                img = img.resize((img_size, img_size))
                img.save(os.path.join(dataset_path, folder, subfolder, image))

resize_images(yolo_folder_path)

# Train YOLO

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n-cls.pt")  # load a pretained model

model.train(data=yolo_folder_path, epochs=50, rect=True, device=device)  # train the model